In [3]:
import pyodbc
from datetime import datetime
import os
import shutil
import win32com.client as win32

class FileValidator:
    def __init__(self, server, database, user, password):
        # Inicializando a classe com parâmetros de conexão
        self.connection_string = (
            f"DRIVER={{ODBC Driver 17 for SQL Server}};"
            f"SERVER={server};"
            f"DATABASE={database};"
            f"UID={user};"
            f"PWD={password}"
        )
        self.conn = None
        self.cursor = None

    def connect_to_db(self):
        # Estabelece a conexão com o banco de dados
        try:
            self.conn = pyodbc.connect(self.connection_string)
            self.cursor = self.conn.cursor()
        except pyodbc.Error as e:
            sqlstate = e.args[0]
            message = e.args[1]
            print(f"Erro SQL: {sqlstate} - Mensagem: {message}")

    def close_connection(self):
        # Fecha a conexão com o banco de dados
        if self.cursor:
            self.cursor.close()
        if self.conn:
            self.conn.close()

    def execute_query(self, query):
        # Executa a query SQL e retorna os resultados
        try:
            self.cursor.execute(query)
            return self.cursor.fetchall()
        except pyodbc.Error as e:
            sqlstate = e.args[0]
            message = e.args[1]
            print(f"Erro SQL: {sqlstate} - Mensagem: {message}")
            return []

    def func_valida_existencia_arquivo(self, subsection_name, bank):
        print(f"Iniciando processo para: {subsection_name}")
        # Caminho do arquivo modelo
        path_model = r"\\172.17.1.115\Temp\\BookToPy\\Modelos\\Book - Modelo.xlsb"

        # Valida se o arquivo existe
        if os.path.exists(path_model):
            current_year = datetime.now().year
            year_str = str(current_year)
            path_model_to_year = rf"\\172.17.1.115\Temp\\BookToPy\\{year_str}"

            print('0.1 - O arquivo modelo existe!')

            if not os.path.exists(path_model_to_year):
                os.makedirs(path_model_to_year)
                print('02. Pasta do ano criada!')

            month_number = datetime.now().month
            meses_em_portugues = [
                '01.Janeiro', '02.Fevereiro', '03.Março', '04.Abril', '05.Maio', '06.Junho',
                '07.Julho', '08.Agosto', '09.Setembro', '10.Outubro', '11.Novembro', '12.Dezembro'
            ]
            mes_atual = meses_em_portugues[month_number - 1]
            path_model_to_month = rf"\\172.17.1.115\Temp\\BookToPy\\{year_str}\\{mes_atual}\\{bank}\\{subsection_name}"

            if not os.path.exists(path_model_to_month):
                os.makedirs(path_model_to_month)
                print('0.3 - Pasta do mês criada!')

            # Caminho do novo arquivo com o nome do subsection_name
            path_param = rf"{path_model_to_month}\\{subsection_name}.xlsb"
            
            # Copia o arquivo modelo para o novo caminho com o nome do subsection_name
            if not os.path.exists(path_param):
                shutil.copy(path_model, path_param)
                print(f'0.4 - Arquivo {subsection_name}.xlsb criado')
            else:
                print(f'0.4 - Arquivo {subsection_name}.xlsb já existe')

            return path_param 
        else:
            print('Arquivo ************MODELO************ não existe!')
            return None


    def update_macros(self, path_to_file, result_distinct,schemaBook,param_schemaBookUnique):
        print(f"0.6 - Atualizando macros para o arquivo: {path_to_file}")
        
        # Verifica se o arquivo existe
        if not os.path.exists(path_to_file):
            print(f"Arquivo não encontrado: {path_to_file}")
            return
        
        excel = None
        workbook = None  #Inicializa workbook fora do try-except
        
        try:
            # Inicia uma instância do Excel
            excel = win32.Dispatch("Excel.Application")

            excel.Visible = False 

            # Abre o arquivo Excel
            workbook = excel.Workbooks.Open(Filename=path_to_file)
            print("0.7 - Arquivo Excel aberto.")

            # Habilita as macros (segurança baixa)
            excel.AutomationSecurity = 1

            # Atualiza as células com valores do `result_distinct`
            sheet = workbook.Sheets('Daily')
            print("0.8 - Sheet 'Daily' encontrada.")

            # ID dos Meses
            sheet.Range("P1").Value = result_distinct["idMesRef1"]
            sheet.Range("Q1").Value = result_distinct["idMesRef2"]
            sheet.Range("R1").Value = result_distinct["idMesRef3"]

            # ID dos Dias
            sheet.Range("T1").Value = result_distinct["idDiaRef1"]
            sheet.Range("V1").Value = result_distinct["idDiaRef2"]
            sheet.Range("X1").Value = result_distinct["idDiaRef3"]

            # Tabela do book 
            sheet_schemaBook = workbook.Sheets('Consulta')
            sheet_schemaBook.Range("B2").Value = schemaBook
            sheet_schemaBook.Range("B41").Value = param_schemaBookUnique

            print("0.9 - Células atualizadas.")

            # Executa a macro
            excel.Application.Run('Atu_All.Atu_All')
            print("0.10 - Macro executada.")

            # Salva e fecha o arquivo
            workbook.Save()
            print("0.11 - Arquivo salvo.")

        except Exception as e:
            print(f"Erro ao atualizar as macros: {e}")
        
        finally:
            # Verifica se o workbook foi inicializado antes de tentar fechá-lo
            if workbook:
                workbook.Close(SaveChanges=1)  # Salva e fecha o arquivo
            if excel:
                excel.Application.Quit()  # Fecha o Excel de forma segura

    def validate_and_process_files(self):
        # Query para obter as carteiras
        query_validacao = """
                                select distinct
                                        b.Banco,
                                        a.Carteira,
                                        a.idDaily,
                                        a.validadorFinal,
                                        'Book - ' + a.Carteira fileName,
                                        a.param_schemaBook,
                                        replace(a.param_schemaBook,'FunilBook','FunilUniqueBook') param_schemaBookUnique

                                from dw.glo.MapaBookPesoToNext a
                                join dw.glo.dp_CarteirasToNext b ON a.idDaily = b.idDaily
                                where CONVERT(DATE, DataAtualizacao) = convert(DATE, GETDATE())
                                and a.Carteira <> 'Solar'
                            """

        # Query para obter as IDs das posições no Excel
        query_ids_das_posicoes_no_excel = """
            SELECT idMesRef1, idMesRef2, idMesRef3,
                   idDiaRef1, idDiaRef2, idDiaRef3
            FROM wr_topy.api.RetornoPosicoesIdExcel
            WHERE CONVERT(DATE, DataAtualizacao) = CONVERT(DATE, GETDATE())
        """

        # Conecta ao banco
        self.connect_to_db()

        # Dicionário para armazenar as linhas
        resultados_distinct = {}

        try:
            # Executa a query para obter as IDs das posições no Excel
            self.cursor.execute(query_ids_das_posicoes_no_excel)
            for row in self.cursor.fetchall():
                resultados_distinct = {
                    "idMesRef1": row.idMesRef1,
                    "idMesRef2": row.idMesRef2,
                    "idMesRef3": row.idMesRef3,
                    "idDiaRef1": row.idDiaRef1,
                    "idDiaRef2": row.idDiaRef2,
                    "idDiaRef3": row.idDiaRef3
                }

            # Executa a query para obter as carteiras
            rows_validacao_carteira = self.execute_query(query_validacao)
            for row in rows_validacao_carteira:
                banco = row[0]
                path = row[4]
                param_schemaBook = row[5]
                param_schemaBookUnique = row[6]
                                
                path_model_to_month = self.func_valida_existencia_arquivo(path,banco)
                    
                    # Se o arquivo foi copiado com sucesso, atualiza as macros
                if path_model_to_month:
                    self.update_macros(path_model_to_month, resultados_distinct,param_schemaBook,param_schemaBookUnique)
                else:
                    print(f"i gotta have problem here vsf")

        finally:
            # Fecha a conexão
            self.close_connection()

# Instanciando e chamando o método de validação
file_validator = FileValidator(
    server="172.17.1.35", 
    database="WR_ToPy", 
    user="USR_MIS", 
    password="MIS@321"
)

# Chamando o método para validar e processar os arquivos
file_validator.validate_and_process_files()


In [ ]:
import win32com.client as win32
import os


def updateMacros(pathToFile):
    # Verifica se o arquivo existe
    if not os.path.exists(pathToFile):
        print(f"Arquivo não encontrado: {pathToFile}")
        return
    
    excel = None
    workbook = None  # Inicializa workbook fora do try-except
    
    try:
        # Inicia uma instância do Excel
        excel = win32.Dispatch("Excel.Application")
        excel.Visible = True  # Defina como False se não quiser mostrar o Excel

        # Abre o arquivo Excel
        workbook = excel.Workbooks.Open(Filename=pathToFile)

        # Habilita as macros (segurança baixa)
        excel.AutomationSecurity = 3  # "3" permite execução de macros sem bloqueio

        # Atualiza as células
        sheet = workbook.Sheets('Daily')

        # ID dos Meses
        sheet.Range("P1").Value = 58
        sheet.Range("Q1").Value = 59
        sheet.Range("R1").Value = 60

        # ID dos Dias
        sheet.Range("T1").Value = 18
        sheet.Range("V1").Value = 21
        sheet.Range("X1").Value = 18

        # Executa a macro
        # Verifique se o nome da macro e do módulo estão corretos
        excel.Application.Run('Atu_All.Atu_All')

        # Salva e fecha o arquivo
        workbook.Save()

    except Exception as e:
        print(f"Erro: {e}")
    
    finally:
        # Verifica se o workbook foi inicializado antes de tentar fechá-lo
        if workbook:
            workbook.Close(SaveChanges=1)  # Salva e fecha o arquivo
        if excel:
            excel.Application.Quit()  # Fecha o Excel de forma segura

# Caminho do arquivo
file_path = r"\\172.17.1.115\Temp\BookToPy\2024\12.Dezembro\BV\Book - CL\Book - CL.xlsb"

# Chama a função
updateMacros(file_path)


In [82]:
import pyodbc

# Variáveis globais
relationIds_ToVBA = {
    "idMesRef1": 0,
    "idMesRef2": 0,
    "idMesRef3": 0,
    "idDiaRef1": 0,
    "idDiaRef2": 0,
    "idDiaRef3": 0
}

def returnIds():
    connection_string = (
        f"DRIVER={{ODBC Driver 17 for SQL Server}};"
        f"SERVER=172.17.1.35;"
        f"DATABASE=WR_ToPy;"
        f"UID=USR_MIS;"
        f"PWD=MIS@321"
    )
    
    conn = pyodbc.connect(connection_string)
    cursor = conn.cursor()
    
    # Consulta SQL
    query = """ 
    select 
        idMesRef1,
        idMesRef2,
        idMesRef3,
        idDiaRef1,
        idDiaRef2,
        idDiaRef3
    from WR_ToPy.api.tbdp
    where convert(date, DataAtualizacao) = convert(date, getdate())
    """
    cursor.execute(query)
    
    # Preenche o dicionário com os dados da query
    row = cursor.fetchone() 
    if row:
        relationIds_ToVBA["idMesRef1"] = row.idMesRef1
        relationIds_ToVBA["idMesRef2"] = row.idMesRef2
        relationIds_ToVBA["idMesRef3"] = row.idMesRef3
        relationIds_ToVBA["idDiaRef1"] = row.idDiaRef1
        relationIds_ToVBA["idDiaRef2"] = row.idDiaRef2
        relationIds_ToVBA["idDiaRef3"] = row.idDiaRef3

    cursor.close()
    conn.close()
returnIds()


print(relationIds_ToVBA)


{'idMesRef1': 58, 'idMesRef2': 59, 'idMesRef3': 60, 'idDiaRef1': 21, 'idDiaRef2': 24, 'idDiaRef3': 22}
